# Carga de JSON a CSV

Este método es válido cuando **todos los datos** a extraer se encuentran **en el mismo nivel de anidamiento**.

Si los datos pueden estar a distintos niveles, hay que utilizar el método de `aplanamiento.py`

In [19]:
import json
import pandas as pd

In [20]:
# Apertura del archivo
carpeta_datos = "./datos/"
f = open(carpeta_datos + '11369319.json')

# Conversión en diccionario
data = json.load(f)
# Cierre del archivo
f.close()

In [21]:
print(data)

{'statistics': [{'period': 'ALL', 'groups': [{'groupName': 'Expected', 'statisticsItems': [{'name': 'Expected goals', 'home': '3.20', 'away': '1.81', 'compareCode': 1, 'statisticsType': 'positive', 'valueType': 'event', 'homeValue': 3.2, 'awayValue': 1.81}, {'name': 'Expected assists', 'home': '2.60', 'away': '1.07', 'compareCode': 1, 'statisticsType': 'positive', 'valueType': 'event', 'homeValue': 2.6, 'awayValue': 1.07}]}, {'groupName': 'Possession', 'statisticsItems': [{'name': 'Ball possession', 'home': '56%', 'away': '44%', 'compareCode': 1, 'statisticsType': 'positive', 'valueType': 'event', 'homeValue': 56, 'awayValue': 44}]}, {'groupName': 'Shots', 'statisticsItems': [{'name': 'Total shots', 'home': '23', 'away': '15', 'compareCode': 1, 'statisticsType': 'positive', 'valueType': 'event', 'homeValue': 23, 'awayValue': 15}, {'name': 'Shots on target', 'home': '10', 'away': '5', 'compareCode': 1, 'statisticsType': 'positive', 'valueType': 'event', 'homeValue': 10, 'awayValue': 5},

Normalize no nos satisface

In [22]:
df = pd.json_normalize(data, sep='_')
df.head()

,statistics
0,"[{'period': 'ALL', 'groups': [{'groupName': 'E..."


La información se encuentra en una única clave dentro del diccionario llamada statistics

In [23]:
data = data["statistics"]

Dentro de data, ahora hay una lista de tres elementos. 
Tomamos el primero, que corresponde al partido completo. Los otros dos son para el primer y segundo tiempo.

In [24]:
all = data[0]["groups"]

Ahora json-normalize hace mejor trabajo

In [25]:
json_all =json.dumps(all)
df = pd.json_normalize(json.loads(json_all), sep='_')
print(df)

     groupName                                    statisticsItems
0     Expected  [{'name': 'Expected goals', 'home': '3.20', 'a...
1   Possession  [{'name': 'Ball possession', 'home': '56%', 'a...
2        Shots  [{'name': 'Total shots', 'home': '23', 'away':...
3       TVData  [{'name': 'Corner kicks', 'home': '6', 'away':...
4  Shots extra  [{'name': 'Big chances', 'home': '7', 'away': ...
5       Passes  [{'name': 'Passes', 'home': '530', 'away': '40...
6        Duels  [{'name': 'Dribbles', 'home': '6/9 (67%)', 'aw...
7    Defending  [{'name': 'Tackles', 'home': '12', 'away': '9'...


La columna statisticsItems contiene una lista de listas. Si alineo los elementos en una sola lista unidimensional...

In [9]:
statistics_items = df["statisticsItems"].tolist()
statistics_items

items = []
for elemento in statistics_items:
    items.extend(elemento)
print(items)

[{'name': 'Expected goals', 'home': '3.20', 'away': '1.81', 'compareCode': 1, 'statisticsType': 'positive', 'valueType': 'event', 'homeValue': 3.2, 'awayValue': 1.81}, {'name': 'Expected assists', 'home': '2.60', 'away': '1.07', 'compareCode': 1, 'statisticsType': 'positive', 'valueType': 'event', 'homeValue': 2.6, 'awayValue': 1.07}, {'name': 'Ball possession', 'home': '56%', 'away': '44%', 'compareCode': 1, 'statisticsType': 'positive', 'valueType': 'event', 'homeValue': 56, 'awayValue': 44}, {'name': 'Total shots', 'home': '23', 'away': '15', 'compareCode': 1, 'statisticsType': 'positive', 'valueType': 'event', 'homeValue': 23, 'awayValue': 15}, {'name': 'Shots on target', 'home': '10', 'away': '5', 'compareCode': 1, 'statisticsType': 'positive', 'valueType': 'event', 'homeValue': 10, 'awayValue': 5}, {'name': 'Shots off target', 'home': '10', 'away': '5', 'compareCode': 1, 'statisticsType': 'negative', 'valueType': 'event', 'homeValue': 10, 'awayValue': 5}, {'name': 'Blocked shots'

Ya tengo una lista de diccionarios, todos iguales. Si lo convierto en un dataframe ya tengo los datos numéricos en celdas individuales

In [12]:
datos_final = pd.DataFrame(items)
datos_final = datos_final.T
print(datos_final)

                            0                 1                2   \
name            Expected goals  Expected assists  Ball possession   
home                      3.20              2.60              56%   
away                      1.81              1.07              44%   
compareCode                  1                 1                1   
statisticsType        positive          positive         positive   
valueType                event             event            event   
homeValue                  3.2               2.6             56.0   
awayValue                 1.81              1.07             44.0   
homeTotal                  NaN               NaN              NaN   
awayTotal                  NaN               NaN              NaN   

                         3                4                 5              6   \
name            Total shots  Shots on target  Shots off target  Blocked shots   
home                     23               10                10              3 

Convierto la primera fila en nombres de columnas

In [13]:
datos_final.columns = datos_final.iloc[0]
datos_final = datos_final[1:]
print(datos_final)

name           Expected goals Expected assists Ball possession Total shots  \
home                     3.20             2.60             56%          23   
away                     1.81             1.07             44%          15   
compareCode                 1                1               1           1   
statisticsType       positive         positive        positive    positive   
valueType               event            event           event       event   
homeValue                 3.2              2.6            56.0        23.0   
awayValue                1.81             1.07            44.0        15.0   
homeTotal                 NaN              NaN             NaN         NaN   
awayTotal                 NaN              NaN             NaN         NaN   

name           Shots on target Shots off target Blocked shots Corner kicks  \
home                        10               10             3            6   
away                         5                5             5  

Extraigo las dos filas (home y away)

In [14]:
datos_final_home = datos_final.loc["home"]
datos_final_away = datos_final.loc["away"]

Las convierto en DataFrames (las filas, al ser unidimensionales se convierten automáticamente en "Series" y necesito que sean DataFrames para unirlas)

In [15]:
datos_final_home = pd.DataFrame(datos_final_home).transpose()
datos_final_away = pd.DataFrame(datos_final_away).transpose()

Elimino los índices para que al usar join me ponga los datos en la misma fila (fila = 0)

In [16]:
datos_final_home = datos_final_home.reset_index(drop=True)
datos_final_away = datos_final_away.reset_index(drop=True)

...y hago join

In [17]:
final = datos_final_home.join(datos_final_away, lsuffix='_home', how='outer', rsuffix='_away')

In [18]:
# Guardar final en un csv
final.to_csv(carpeta_datos + "final.csv")